In [1]:
import torch
import torch.nn as nn
from utils import get_cifar10_dataloader
from models import ConvNetTrainer
# import efficientnet from torchvision models
from torchvision.models import efficientnet

train, valid, test = get_cifar10_dataloader(512)
model = efficientnet.efficientnet_b0(pretrained=False)

Files already downloaded and verified


/Users/vornao/Desktop/unipi/CL/project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vornao/Desktop/unipi/CL/project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
# change the last layer to fit CIFAR10
# load model from saved weights
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
# try:
#     model.load_state_dict(torch.load('../models/efficientnet_b0.pth'))
# except:
#     print('No weights found for EfficientNet B0')

trainer = ConvNetTrainer(
    model=model, 
    optimizer='AdamW',
    lr=1e-2, 
    weight_decay=1e-6, 
    device='mps', 
    train_loader=train, 
    test_loader=valid
)

Using AdamW
Model created and moved to mps


In [5]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of trainable parameters: {pytorch_total_params}')

Total number of trainable parameters: 4020358


In [6]:
trainer.train(num_epochs=100, earlystopping=True)
# save the model
torch.save(trainer.model.state_dict(), '../models/efficientnet_b0.pth')

 11%|█████                                         | 11/100 [03:57<32:01, 21.59s/it, TL: 0.577, TA: 0.798, VL: 0.862, VA: 0.725]

In [ ]:
# test the model
test_loss, test_acc = trainer.test(test)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')